<a href="https://colab.research.google.com/github/JohnAntonusMaximus/transfer-learning/blob/master/Transfer_Learning_%26_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cat Dog Classifier Using MobileNetV2 Transfer Learning & Fine Tuning


![dog and cat analyzer](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)


### Load Dependencies and Imports

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha

In [0]:
!pip install tqdm

### Get the dataset

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

In [0]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

### Specify the path to the data and unpack the zipfile

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

### Set up directory variables for getting images

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [0]:
train_dir = os.path.join(dataset_path_new,"train")
validation_dir = os.path.join(dataset_path_new,"validation")

### Load the base model, MobileNetV2 and specify the image input shape

In [0]:
IMAGE_SHAPE = (128,128,3)

In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE, include_top=False, weights="imagenet")

### Freeze the base network weights

In [0]:
base_model.trainable = False

In [18]:
base_model.output

<tf.Tensor 'out_relu/Relu6:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

### Create a global average pooling layer to connect a custom output layer (The input shape of the output layer needs to match the previous layer, could flatten the images but they are too large in this case)

In [0]:
global_average_layer

### Specify the custom output layer, since this is binary classification we only need 1 neuron with a sigmoid activation

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(global_average_layer)

### Summarize the base model before we connect our custom layers

In [0]:
base_model.summary()

### Connect our pre-trained model with our custom output layer

In [0]:
model = tf.keras.models.Model(inputs=base_model.inputs, outputs=prediction_layer)

### Notice our model now has the custom output layer

In [0]:
model.summary()

### Compile the model with our optimizer (RMSprop is best for MobileNet), Binary crossentropy for our loss and simple accuracy metric

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.0001),loss="binary_crossentropy", metrics=["accuracy"])

### Images need to be rescaled before being fed into the model

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255,)
data_gen_valid = ImageDataGenerator(rescale=1/255,)

### Use the Flow From Directory to get images, resize them and feed them into the network

In [0]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

In [0]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

###Train the model, need to use the fit_generator function as opposed to just fit because we are using data generators - This will take a little while because our network is large

In [0]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [34]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9409999847412109


## Fine Tuning of Transfer Learning

In [0]:
### We're going to unfreeze the few tops layers of the base network, so we can get some better accuracy on our network as a whole 

In [37]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

Number of layers in the base model: 155


### Unfreeze the base model layers

In [0]:
base_model.trainable = True

### Create an index that specificies what layers in the base model you want to train

In [0]:
fine_tune_at = 100

### Create a For Loop that freezes all layers after the fine tuning index

In [0]:
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

### Recompile the model

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [0]:
model.fit_generator(train_generator, epochs=2,validation_data=valid_generator)

### In this case, fine tuning did not help our model, we have overfitting to the training data because our dataset is too small

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [0]:
print("Accuracy after fine tuning: {}".format(valid_accuracy))